## Best way to clean dataset for now:
    
unit: pair 

number of stds as threshold: 0.5

number of annotations:

total: 145935 
remaining: 125390 (0.8592181450645835)


IAA:
total: 0.3620784770469727 
level of pairs: 0.5934617635269503

In [1]:
from load_data import load_experiment_data
from utils_analysis import sort_by_key
from clean_annotations import clean_workers
#from utils_analysis import load_analysis
from aggregation import aggregate_binary_labels
#from calculate_iaa import load_rel_level_mapping
from calculate_iaa import get_agreement

from collections import defaultdict, Counter

In [2]:
def get_stats(data_dict_list):
    
    stats_dict = dict()
    #remove tests
    data_dict_list = [d for d in data_dict_list if not d['property'].startswith('_')]
    
    n_annotations = len(data_dict_list)
    annotations_by_worker = sort_by_key(data_dict_list, ['workerid'])
    annotations_by_pair = sort_by_key(data_dict_list, ['property', 'concept'])
    annotations_by_property = sort_by_key(data_dict_list, ['property'])
    concepts = set([d['concept'] for d in data_dict_list])
    iaa_labels = get_agreement(data_dict_list, collapse_relations = False, disable_kappa = True, v=False)
    iaa_levels = get_agreement(data_dict_list, collapse_relations = 'levels', disable_kappa = True, v=False)
    annotations_by_triple = sort_by_key(data_dict_list, ['relation','property', 'concept'])
    n_workers = len(annotations_by_worker)
    n_triples = len(annotations_by_triple)
    
    stats_dict['n_annotations'] = n_annotations
    stats_dict['n_properties'] = len(annotations_by_property)
    stats_dict['n_pairs'] = len(annotations_by_pair)
    stats_dict['n_workers'] = n_workers
    stats_dict['n_triples'] = n_triples
    stats_dict['n_workers_per_triple'] = n_annotations/n_triples
    stats_dict['annotations_per_worker_av'] = n_annotations/n_workers
    stats_dict['iaa_label'] = iaa_labels['Krippendorff']
    stats_dict['iaa_level'] = iaa_levels['Krippendorff']
    
    return stats_dict
    
    #get_agreement(dict_list_out, collapse_relations = False, v=True, disable_kappa=False)

In [3]:


run = '1'
batch = '*'
n_q = '*'
group = 'experiment1'

data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
stats1 = get_stats(data_dict_list)

run = '3'
batch = '*'
n_q = '*'
group = 'experiment1'

data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
stats2 = get_stats(data_dict_list)

run = '4'
batch = '*'
n_q = '*'
group = 'experiment2'

data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
stats3 = get_stats(data_dict_list)


run = '*'
batch = '*'
n_q = '*'
group = 'experiment*'

data_dict_list = load_experiment_data(run, group, n_q, batch, remove_not_val = True)
stats_total = get_stats(data_dict_list)

Discarded 0.0 annotations.
Discarded 655.0 annotations.
Discarded 0.0 annotations.
Discarded 655.0 annotations.


In [4]:
# create_table of all 
import pandas as pd

stats_dict = dict()
stats_dict['total'] = stats_total
stats_dict['version1'] = stats1
stats_dict['version2'] = stats2
stats_dict['version3'] = stats3

columns = dict()
columns['data'] = list(stats1.keys())

for n, s in stats_dict.items():
    vals = list(s.values())
    columns[n] = vals
    
df = pd.DataFrame.from_dict(columns).round(2)
df

,data,total,version1,version2,version3
0,n_annotations,164220.00,20971.00,41447.00,101802.00
1,n_properties,13.00,3.00,3.00,10.00
2,n_pairs,1538.00,425.00,426.00,1104.00
3,n_workers,1068.00,285.00,547.00,439.00
4,n_triples,14763.00,4105.00,4094.00,10068.00
5,n_workers_per_triple,11.12,5.11,10.12,10.11
6,annotations_per_worker_av,153.76,73.58,75.77,231.90
7,iaa_label,0.30,0.23,0.21,0.35
8,iaa_level,0.50,0.41,0.33,0.54


In [5]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrr}
\toprule
                      data &      total &  version1 &  version2 &  version3 \\
\midrule
             n\_annotations &  162000.00 &  20971.00 &  41447.00 &  99582.00 \\
              n\_properties &      13.00 &      3.00 &      3.00 &     10.00 \\
                   n\_pairs &    1521.00 &    425.00 &    426.00 &   1087.00 \\
                 n\_workers &    1068.00 &    285.00 &    547.00 &    439.00 \\
                 n\_triples &   14615.00 &   4105.00 &   4094.00 &   9920.00 \\
      n\_workers\_per\_triple &      11.08 &      5.11 &     10.12 &     10.04 \\
 annotations\_per\_worker\_av &     151.69 &     73.58 &     75.77 &    226.84 \\
                 iaa\_label &       0.30 &      0.23 &      0.21 &      0.35 \\
                 iaa\_level &       0.50 &      0.41 &      0.33 &      0.54 \\
\bottomrule
\end{tabular}



In [ ]:
# Clean annotations
unit = 'batch'
n_stds = 0.5
data_dict_list_clean = clean_worker_cont_rate(data_dict_list, run,  group,  batch, unit, n_stds)

In [6]:
def get_pair_table(pair_label_dicts, vote = 'majority_vote', label = 'level'):
    
    #all_levels = set([d[vote] for d in pair_label_dicts])
    prop_pair_dict = defaultdict(list)
    for pd in pair_label_dicts:
        pair = pd['pair'].strip()
        prop = pair.split('-')[0].strip()
        prop_pair_dict[prop].append(pd)
        
    prop_dicts = []
    for prop, pds in prop_pair_dict.items():
        all_labels_prop = []
        dicts_by_pair = sort_by_key(pds, ['pair'])
        for pair, pair_dicts in dicts_by_pair.items():
            labels_total = [d[label] for d in pair_dicts]
            labels_true = [d[label] for d in pair_dicts if d[vote] == True]
            if labels_true != []:
                labels_true_str = '-'.join(sorted(set(labels_true)))
            else:
                labels_true_str = 'no_true_label'
                #print(pair)
                #print([d[vote] for d in pair_dicts])
            all_labels_prop.append(labels_true_str)
        prop_dict = dict()
        prop_dict['property'] = prop
        prop_dict['total'] = len(dicts_by_pair)
        labels = set(all_labels_prop)
        for l in labels:
            prop_dict[l] = all_labels_prop.count(l)
        prop_dicts.append(prop_dict)
    return prop_dicts

pair_table = get_pair_table(labels, vote = 'top_vote')

NameError: name 'labels' is not defined

In [8]:
import pandas as pd

# Raw data
labels  = aggregate_binary_labels(data_dict_list)

pair_table_raw = get_pair_table(labels, vote = 'top_vote')
#print(labels[0])
df = pd.DataFrame(pair_table_raw)
columns = ['property', 'total', 'all', 'all-some', 'some',
           'few-some', 'few', 'creative-few', 'creative', #'no_true_label',
           'all-creative', 'all-few', 'all-creative-few', 'creative-few-some']
#columns = ['property', 'total', 'typical_of_concept', 'typical_of_property']
#total_pairs = sum(df['total'])
#print(f'#pairs {total_pairs}')
#print(top_labels[0])
df = df[columns].sort_values('property').fillna(0)
df

,property,total,all,all-some,some,few-some,few,creative-few,creative,all-creative,all-few,all-creative-few,creative-few-some
7,black,117,7,4.0,54.0,5.0,46,0.0,1.0,0.0,0.0,0.0,0.0
8,dangerous,106,47,4.0,9.0,4.0,39,2.0,1.0,0.0,0.0,0.0,0.0
2,fly,120,49,0.0,4.0,0.0,54,8.0,4.0,0.0,0.0,0.0,1.0
4,hot,112,31,12.0,35.0,2.0,27,3.0,2.0,0.0,0.0,0.0,0.0
1,lay_eggs,112,59,0.0,0.0,0.0,47,0.0,3.0,0.0,3.0,0.0,0.0
3,made_of_wood,110,23,9.0,41.0,6.0,27,1.0,3.0,0.0,0.0,0.0,0.0
11,red,174,11,4.0,81.0,11.0,59,0.0,7.0,0.0,0.0,0.0,0.0
10,roll,120,34,3.0,41.0,6.0,27,0.0,3.0,1.0,3.0,0.0,1.0
12,round,140,36,8.0,72.0,0.0,18,0.0,5.0,0.0,0.0,0.0,0.0
5,square,103,11,12.0,58.0,2.0,20,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
print(df.to_latex(index = False))

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
     property &  total &  all &  all-some &  some &  few-some &  few &  creative-few &  creative &  all-creative &  all-few &  all-creative-few &  creative-few-some \\
\midrule
        black &    117 &    7 &       4.0 &  54.0 &       5.0 &   46 &           0.0 &       1.0 &           0.0 &      0.0 &               0.0 &                0.0 \\
    dangerous &    106 &   47 &       4.0 &   9.0 &       4.0 &   39 &           2.0 &       1.0 &           0.0 &      0.0 &               0.0 &                0.0 \\
          fly &    120 &   49 &       0.0 &   4.0 &       0.0 &   54 &           8.0 &       4.0 &           0.0 &      0.0 &               0.0 &                1.0 \\
          hot &    112 &   31 &      12.0 &  35.0 &       2.0 &   27 &           3.0 &       2.0 &           0.0 &      0.0 &               0.0 &                0.0 \\
     lay\_eggs &    112 &   59 &       0.0 &   0.0 &       0.0 &   47 &           0.0 &       3.0 &           0

In [84]:
# clean data
top_labels_clean = aggregate_binary_labels(data_dict_list_clean)
pair_table_clean = get_pair_table(top_labels_clean, vote = 'majority_vote')
df = pd.DataFrame(pair_table_clean)
columns = ['property', 'total', 'all', 'all-some', 'some',
           'few-some', 'few', 'creative-few', 'creative', 'no_true_label',
           'all-creative', 'all-few', 'all-creative-few', 'creative-few-some']
#columns = ['property', 'total', 'typical_of_concept', 'typical_of_property']
#total_pairs = sum(df['total'])
#print(f'#pairs {total_pairs}')
#print(top_labels[0])
df[columns].sort_values('property').fillna(0)

,property,total,all,all-some,some,few-some,few,creative-few,creative,no_true_label,all-creative,all-few,all-creative-few,creative-few-some
0,black,114,2,24,20.0,35.0,24,2,0.0,3,0.0,0.0,0.0,3.0
5,dangerous,105,25,29,2.0,12.0,21,8,0.0,2,0.0,0.0,0.0,5.0
3,fly,117,42,4,3.0,1.0,27,32,2.0,6,0.0,0.0,0.0,0.0
1,hot,109,9,48,10.0,13.0,11,8,0.0,2,0.0,0.0,0.0,5.0
11,lay_eggs,109,48,6,0.0,0.0,35,12,1.0,7,0.0,0.0,0.0,0.0
2,made_of_wood,108,4,47,14.0,19.0,17,3,1.0,2,0.0,0.0,0.0,1.0
9,red,174,1,37,32.0,28.0,38,17,1.0,2,0.0,0.0,0.0,14.0
7,roll,120,12,19,3.0,10.0,11,5,0.0,5,1.0,8.0,4.0,1.0
8,round,140,12,56,21.0,19.0,5,2,4.0,10,0.0,0.0,0.0,3.0
6,square,100,1,45,27.0,4.0,12,4,0.0,5,0.0,0.0,0.0,1.0


In [95]:
# get statistics:

print(f'# annotations {len(data_dict_list)}')
print(f'# properties {len(pair_table_raw)}')
print(f"# pairs {len(sort_by_key(data_dict_list, ['property', 'concept']))}")
print(f'# concepts per property \
{sum([d["total"]for d in pair_table_raw])/len(pair_table_raw)}')

# annotations 163987
# properties 13
# pairs 1507
# concepts per property 115.15384615384616
